# Flux DreamBooth Training for Portrait Images

This notebook will help you train a personalized Flux model using DreamBooth technique on your portrait images.

## Setup and Dependencies

In [1]:
# Install base dependencies
!pip install --upgrade pip
!pip install accelerate transformers torch torch-fidelity mediapy wandb
!pip install scipy ftfy
!pip install -q triton prodigyopt
!pip install -q peft>=0.6.0

# Install diffusers from source
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .
%cd examples/dreambooth
!pip install -r requirements_flux.txt

# Install xformers last to ensure compatibility
!pip install xformers --force-reinstall


ok
y
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparselt_cu12-0.6

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparselt_cu12-0.6.2-py3

In [2]:
# Install torchvision
!pip install torchvision

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparselt_cu12-0.6.2-py3

In [1]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

PyTorch version: 2.6.0+cu124
Torchvision version: 0.21.0+cu124


In [1]:
!pip uninstall transformers -y
!pip install transformers==4.35.0
import transformers
print("Transformers version:", transformers.__version__)

Found existing installation: transformers 4.35.0
Uninstalling transformers-4.35.0:
  Successfully uninstalled transformers-4.35.0
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.17.3 which is incompatible.
sentence-transformers 4.1.0 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.17.3 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.0 which is incompatible.
Transformers version: 4.35.0


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
from transformers import AutoImageProcessor
print("Import successful!")

Import successful!


In [3]:
import PIL
import numpy
print("Pillow version:", PIL.__version__)
print("NumPy version:", numpy.__version__)

Pillow version: 11.2.1
NumPy version: 2.2.6


In [9]:
!pip uninstall xformers -y
!git clone https://github.com/facebookresearch/xformers.git
%cd xformers
!pip install -e .

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'xformers': No such file or directory
[Errno 2] No such file or directory: 'xformers'
/content/xformers
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [8]:
from diffusers import AutoencoderKL
print("Import successful!")

ImportError: cannot import name 'HF_HOME' from 'huggingface_hub.constants' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/constants.py)

## Install Diffusers from Source
As recommended in the documentation, we'll install diffusers from source to ensure compatibility.

## Login to Hugging Face
You need to login to access the gated Flux model.

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Configure Accelerate
Set up a basic accelerate configuration for training.

In [4]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

## Set up Dataset Directory
Create a directory for your portrait images and upload them.

In [5]:
from google.colab import files
import os

# Create directory for your images
INSTANCE_DIR = "portrait_images"
os.makedirs(INSTANCE_DIR, exist_ok=True)

# Upload your images
uploaded = files.upload()

# Save uploaded images to the instance directory
for filename, content in uploaded.items():
    with open(os.path.join(INSTANCE_DIR, filename), 'wb') as f:
        f.write(content)

Saving cookboei.jpg to cookboei.jpg


## Training Configuration
Set up the training parameters for DreamBooth LoRA training.

In [ ]:
#Uncomment to unzip your checkpoint from the first phase i.e Phase 1
#!unzip <your_model_checkpoint.zip>

## Launch Training
Start the DreamBooth LoRA training process with optimized parameters for portraits.

In [6]:
# Set training configuration
import os
MODEL_NAME = "black-forest-labs/FLUX.1-schnell"
#INSTANCE_DIR = "portrait_images"
OUTPUT_DIR = "latest-flux-your-identifier"
INSTANCE_PROMPT = "a photo of <your-identifier> person"  # Replace with your identifier

#Uncomment these lines when training a custom checkpoint otherwise, not

#!mkdir -p $OUTPUT_DIR
#!cp <your_model_checkpoint>/checkpoint-500/pytorch_lora_weights.safetensors $OUTPUT_DIR/
#!cp <your_model_checkpoint>/checkpoint-500/optimizer.bin $OUTPUT_DIR/
#!cp <your_model_checkpoint>/checkpoint-500/scheduler.bin $OUTPUT_DIR/
#!cp <your_model_checkpoint>/checkpoint-500/random_states_0.pkl $OUTPUT_DIR/


In [7]:


# Launch training with disabled validation
!accelerate launch train_dreambooth_lora_flux.py \
  --pretrained_model_name_or_path="$MODEL_NAME" \
  --instance_data_dir="$INSTANCE_DIR" \
  --output_dir="$OUTPUT_DIR" \
  --mixed_precision="bf16" \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=256 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=8 \
  --learning_rate=1.0 \
  --optimizer="prodigy" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --seed=42 \
  --gradient_checkpointing \
  --cache_latents \
  --lora_layers="attn.to_k,attn.to_q,attn.to_v,attn.to_out.0" \
  --rank=4

2025-06-04 08:50:41.789145: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 08:50:41.806713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749027041.828090    5380 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749027041.834626    5380 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 08:50:41.856059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Save the Trained Model
After training completes, save your model locally and optionally push to Hugging Face Hub.

In [ ]:
# Download the trained model
!zip -r <your_model_checkpoint>.zip "$OUTPUT_DIR"
files.download('<your_model_checkpoint.zip')